In [ ]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

def crawl(code, time):
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

    date = time[:8]
    
    result = list()
    pages = range(1, 41)
    for page in pages:
        url = 'https://finance.naver.com/item/sise_time.nhn?code={}&thistime={}&page={}'.format(code, time, page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        
        result.extend([(date, t, p) for t, p in zip([i.text for i in dom.select('tr> td:nth-child(1) > span')], [i.text for i in dom.select('tr> td:nth-child(2) > span')])])
    
    return result

In [2]:
conn = sqlite3.connect('Stock.db')
cur = conn.cursor()

# cur.executescript('''
#             DROP TABLE IF EXISTS stock;
#             CREATE TABLE stock(
#                 pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
#                 code TEXT NOT NULL,
#                 date TEXT NOT NULL,
#                 time TEXT NOT NULL,
#                 price TEXT NOT NULL
#             );
#         ''')

codes = {'336260', '009830', '086520', '011930', '018000', '035720', '035420', '036570', '041140', '018260', '005930', '000660', '011790', '014680', '000990', '023530', '004170', '008770', '028260', '026960', '105560', '071050', '055550', '316140', '086790', '047080', '089890', '030350', '230980', '069410'}

data = {i:[] for i in codes}

now = datetime.datetime.now()
year = now.year
month = now.month
day = now.day

if cur.execute('SELECT date FROM stock ORDER BY date DESC').fetchone():
    seen_date = cur.execute('SELECT date FROM stock ORDER BY date DESC').fetchone()[0]
else:
    seen_date = None


for i in tqdm(range(16)):
    now = datetime.datetime(year, month, day)
    if now.weekday() == 5 or now.weekday() == 6:
        pass
    else:
        if month < 10:
            if day < 10:
                time = str(year) + '0' + str(month) + '0' + str(day) + '161122'
            else:
                time = str(year) + '0' + str(month) + str(day) + '161122'
        else:
            if day < 10:
                time = str(year) + str(month) + '0' + str(day) + '161122'
            else:
                time = str(year) + str(month) + str(day) + '161122'
        
        # 만약 해당 날짜가 DB에 있으면, 그 이전 날짜의 데이터도 모두 있는 것이므로 break
        if int(time[:8]) <= int(seen_date):
            break
        else:
            for code in codes:
                result = crawl(code, time)
                for date, t, price in result:
                    cur.execute('INSERT INTO stock(code, date, time, price) VALUES(?,?,?,?)',
                                                         [code, date, t, price])
                    conn.commit()
#         for code in codes:
#             data[code].extend(crawl(code, time))
        
    day -= 1

 69%|███████████████████████████████████████████████████████▋                         | 11/16 [20:07<09:09, 109.81s/it]
